In [52]:
#import *
import numpy as np
import os
import nglview as ngl
from ase.io import read, write
from molmass import Formula
import random

SYSDIR='/home/harish/GroPolBul'
##Give short polymer name and charge
res=input("Polymer abbrevation? (3 or 4 letters and preferable in caps): ") or "PE"
end=input("End group? (3 or 4 letters and preferable in caps): ") or "OH"
chg=input("Charge of the molecule: ") or "0"
print(res,end,chg)
MONDIR=str(SYSDIR)+'/monomers/'+str(poly)+'_'+str(endg)
os.popen('mkdir '+str(MONDIR));os.chdir(str(MONDIR))

##Paramaterize the monomer with acpype
mono=input("Monomer mol2 file or SMILE fomat: ") or "OCCCCCCCCCCCCO"
#os.popen('rm -r '+str(res)+'.acpype')
#os.popen('acpype -i '+str(mono)+' -b '+str(res)+' -n '+str(chg)+' -o gmx').read()
os.popen("sed -e 's/UNL/"+str(res)+"/g' "+str(res)+".acpype/sqm.pdb > "+str(res)+".pdb").read()
os.popen("sed -e 's/UNL/"+str(res)+"/g' "+str(res)+".acpype/ANTECHAMBER_AC.AC > "+str(res)+".ac").read()

#Visualize the molecule
mol=read(str(res)+'.acpype/PE_GMX.gro')
vi=ngl.show_ase(mol);vi.add_label(radius=2,color='black',label_type='atomindex')
vi


Polymer abbrevation? (3 or 4 letters and preferable in caps): 
End group? (3 or 4 letters and preferable in caps): 
Charge of the molecule: 
PE OH 0


mkdir: cannot create directory ‘/home/harish/GroPolBul/monomers/PE_OH’: File exists


Monomer mol2 file or SMILE fomat: 


NGLWidget()

In [53]:
#Defining CHAIN HEAD and TAIL in the monomer directory
os.chdir(str(MONDIR))
ac=open(str(res)+'.ac',mode='r') #Reading .ac file
[next(ac) for _ in range(2)] #Skipping first two lines of text
l=ac.readlines() #Reading lines

#Breaking the parts of monomer to CHAIN, HEAD and TAIL
chain=open(str(res)+'.chain','w+');head=open(str(res)+'.head','w+');tail=open(str(res)+'.tail','w+')
#Atom index where head and tail of monomer
#Check from above ngl view of mol
head_id=int(input("Monomer's Head atom index: ") or 2)
tail_id=int(input("Monomer's Tail atom index: ") or 11)

chain.write('HEAD_NAME '+str(l[head_id].split()[2])+'\n')
tail.write('HEAD_NAME '+str(l[head_id].split()[2])+'\n')
chain.write('TAIL_NAME '+str(l[tail_id].split()[2])+'\n')
head.write('TAIL_NAME '+str(l[tail_id].split()[2])+'\n')

head_omit=input("Atom indices to omit near head: ").split() or [0, 1, 14, 15, 16]
tail_omit=input("Atom indices to omit near tail: ").split() or [12, 13, 37, 38, 39]

for i in range(len(head_omit)):
	chain.write('OMIT_NAME '+str(l[int(head_omit[i])].split()[2])+'\n')
	tail.write('OMIT_NAME '+str(l[int(head_omit[i])].split()[2])+'\n')
chain.write('PRE_HEAD_TYPE '+str(l[tail_id].split()[9])+'\n')
tail.write('PRE_HEAD_TYPE '+str(l[tail_id].split()[9])+'\n')
tail.write('CHARGE '+str(charge))

for i in range(len(tail_omit)):
	chain.write('OMIT_NAME '+str(l[int(tail_omit[i])].split()[2])+'\n')
	head.write('OMIT_NAME '+str(l[int(tail_omit[i])].split()[2])+'\n')
chain.write('POST_TAIL_TYPE '+str(l[head_id].split()[9])+'\n')
chain.write('CHARGE '+str(float(charge)))
head.write('POST_TAIL_TYPE '+str(l[head_id].split()[9])+'\n')
head.write('CHARGE '+str(float(charge)))

chain.close();head.close();tail.close()

#Use prepgen to prepare the CHAIN, HEAD and TAIL res files
#Adds dummy atoms at the desired positions
os.popen('prepgen -i '+str(res)+'.ac -o '+str(res)+'.prepi -f prepi -m '+str(res)+'.chain -rn '+str(res)+' -rf '+str(res)+'.res').read() 
os.popen('prepgen -i '+str(res)+'.ac -o HPT.prepi -f prepi -m '+str(res)+'.head -rn HPT -rf HPT.res').read() 
os.popen('prepgen -i '+str(res)+'.ac -o TPT.prepi -f prepi -m '+str(res)+'.tail -rn TPT -rf TPT.res').read() 


Monomer's Head atom index: 
Monomer's Tail atom index: 
Atom indices to omit near head: 
Atom indices to omit near tail: 


'\nPRE_HEAD_TYPE is    c3\nNet charge of truncated molecule is     0.00\nHEAD_ATOM      3   C1\nMAIN_CHAIN     1    3   C1\nOMIT_ATOM      1    1    O\nOMIT_ATOM      2    2    C\nOMIT_ATOM      3   15    H\nOMIT_ATOM      4   16   H1\nOMIT_ATOM      5   17   H2\nNumber of mainchain atoms (including head and tail atom):     1\nNumber of omited atoms:     5'

In [58]:
#Build the desired homopolymer using above prepi files
n_mono_repeat=int(input("Number of monomers in the repeating unit: ") or 5)
n_mono_pol=int(input("Desired number of monomers in the polymer: ") or 10)
POLDIR=str(SYSDIR)+'/polymers/'+str(res)+'_'+str(end)+'_'+str(n_mono_pol)+'mer'
os.popen('mkdir '+str(POLDIR))
os.chdir(str(POLDIR))
os.popen('cp '+str(MONDIR)+'/'+str(res)+'.pdb '+str(res)+'_repeatunit.pdb')

repeat=" ".join([res] * int(int(n_mono_pol/n_mono_repeat)-2))
print('HPT '+str(repeat)+' TPT')
tleap=open(str(res)+'_tleap.in','w+')
tleap.write('''source leaprc.gaff
loadamberprep '''+str(MONDIR)+'''/'''+str(res)+'''.prepi
loadamberprep '''+str(MONDIR)+'''/HPT.prepi
loadamberprep '''+str(MONDIR)+'''/TPT.prepi
mol = sequence {HPT '''+str(repeat)+''' TPT}
savepdb mol '''+str(res)+'''_polymer.pdb
saveamberparm mol '''+str(res)+'''_polymer.prmtop '''+str(res)+'''_polymer.inpcrd
quit''')
tleap.close()

os.popen('tleap -s -f '+str(res)+'_tleap.in > '+str(res)+'_tleap.out').read()
os.popen('acpype -p '+str(res)+'_polymer.prmtop -x '+str(res)+'_polymer.inpcrd').read()


Number of monomers in the repeating unit: 
Desired number of monomers in the polymer: 15


mkdir: cannot create directory ‘/home/harish/GroPolBul/polymers/PE_OH_15mer’: File exists


HPT PE TPT


'==========================================================================\n| ACPYPE: AnteChamber PYthon Parser interfacE v. 2022.6.6 (c) 2023 AWSdS |\n==========================================================================\nConverting Amber input files to Gromacs ...\n==> Writing GROMACS files\n\n==> Disambiguating lower and uppercase atomtypes in GMX top file, even if identical.\n\n==> Writing GMX dihedrals for GMX 4.5 and higher.\n\nTotal time of execution: less than a second\n'

In [59]:
#Visualize the molecule
mol=read(str(res)+'_polymer.pdb')
vi=ngl.show_ase(mol);#vi.add_label(radius=2,color='black',label_type='atomindex')
vi


NGLWidget()